# Intersection with a crosswalk

**Load dependencies**

In [1]:
rng = MersenneTwister(1);

In [2]:
using POMDPs, POMDPToolbox, DiscreteValueIteration, MDPModelChecking
using StaticArrays
using DeepRL, TensorFlow, DeepQLearning
using AutomotiveDrivingModels, AutomotivePOMDPs
using AutomotiveSensors
using LocalApproximationValueIteration
using Reel, AutoViz
using ProgressMeter
using JLD
using PedCar

In [3]:
include("masking.jl")
include("util.jl")
include("render_helpers.jl")

In [4]:
cam = FitToContentCamera(0.);

## Scenario

In [5]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

## Discrete MDP

In [6]:
mdp = PedCarMDP(env=env, pos_res=2.0, vel_res=2., ped_birth=0.7, car_birth=0.7);
init_transition!(mdp);

In [7]:
@printf("spatial resolution %2.1f m \n", mdp.pos_res)
@printf("pedestrian velocity resolution %2.1f m/s \n", mdp.vel_ped_res)
@printf("car velocity resolution %2.1f m/s \n", mdp.vel_res)
@printf("number of states %d \n", n_states(mdp))
@printf("number of actions %d \n", n_actions(mdp))

spatial resolution 2.0 m 
pedestrian velocity resolution 1.0 m/s 
car velocity resolution 2.0 m/s 
number of states 23456940 
number of actions 4 


In [8]:
vi_data = load("pc_processed.jld")
policy = ValueIterationPolicy(mdp, vi_data["qmat"], vi_data["util"], vi_data["pol"]);

In [12]:
# Load VI data for maksing
state_space = states(mdp);
vi_data = load("pc_util_inter.jld")
@showprogress for s in state_space
    if !s.crash && isterminal(mdp, s)
        si = stateindex(mdp, s)
        vi_data["util"][si] = 1.0
        vi_data["qmat"][si, :] = ones(n_actions(mdp))
    end
end
policy = ValueIterationPolicy(mdp, vi_data["qmat"], vi_data["util"], vi_data["pol"]);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:4739m


In [13]:
JLD.save("pc_processed.jld", "util", policy.util, "qmat", policy.qmat, "pol", policy.policy);

In [9]:
threshold = 0.999
mask = SafetyMask(mdp, policy, threshold);

In [15]:
rand_pol = MaskedEpsGreedyPolicy(mdp, 1.0, mask, rng);
# rand_pol = solve(RandomSolver(), mdp);

In [16]:
hr = HistoryRecorder(rng=rng, max_steps=100)
s0 = initialstate(mdp, rng)
@time hist2 = simulate(hr, mdp, policy, s0);

  1.765905 seconds (448.81 k allocations: 23.336 MiB)


In [17]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
push!(action_hist, UrbanAction(NaN))
duration, fps, render_hist = animate_states(mdp, state_hist, action_hist, mask)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpk61Mt8", 0x0000000000000029, 2.0, nothing)

In [18]:
@showprogress for ep=1:10000
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initialstate(mdp, rng)
    hist2 = simulate(hr, mdp, rand_pol, s0)
    if sum(hist2.reward_hist .< 0.) != 0.
        println("Crash")
        break
    end
end

Progress:  23%|█████████                                |  ETA: 0:18:07

Crash


Progress: 100%|█████████████████████████████████████████| Time: 0:05:26


**Evaluation**

In [19]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(mdp, rand_pol, n_ep=10000, max_steps=100, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress:  99%|█████████████████████████████████████████|  ETA: 0:00:00

 41.613677 seconds (200.58 M allocations: 21.300 GiB, 12.33% gc time)
Summary for 10000 episodes: 
Average reward: 0.042 
Average # of steps: 78.531 
Average # of violations: 0.110 


Progress: 100%|█████████████████████████████████████████| Time: 0:00:41


## Continuous state MDP

In [12]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

In [13]:
# sensor
pomdp = UrbanPOMDP(env=env,
                   ego_goal = LaneTag(2, 1),
                   max_cars=1, 
                   max_peds=1, 
                   car_birth=0.7, 
                   ped_birth=0.7, 
                   obstacles=false, # no fixed obstacles
                   lidar=false,
                   pos_obs_noise = 0., # fully observable
                   vel_obs_noise = 0.,
                   ego_start=20,
                   ΔT=0.5);

In [14]:
function POMDPToolbox.generate_sori(pomdp::UrbanPOMDP, s::Scene, a::UrbanAction, rng::AbstractRNG)
    sp, o, r = generate_sor(pomdp, s, a, rng)
    return sp, o, r, deepcopy(pomdp.models)
end

In [23]:
rand_pol = RandomMaskedPOMDPPolicy(mask, pomdp, rng);
# rand_pol = solve(RandomSolver(), mdp);

In [24]:
hr = HistoryRecorder(rng=rng, max_steps=100)
s0 = initialstate(pomdp,rng)
o0 = generate_o(pomdp, s0, rng)
up = FastPreviousObservationUpdater{UrbanObs}()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0);

  1.573584 seconds (3.23 M allocations: 133.703 MiB, 4.25% gc time)


In [25]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
belief_hist = h.belief_hist
safe_acts = [i[1] for i in h.ainfo_hist]
probas = [i[2] for i in h.ainfo_hist]
routes = [i[3] for i in h.ainfo_hist]

push!(safe_acts, [UrbanAction(NaN)])
push!(probas, [NaN])
push!(routes, PedCar.OFF_ROUTE)
push!(action_hist, UrbanAction(NaN))
duration, fps, render_hist = animate_states(pomdp, state_hist, action_hist, belief_hist, safe_acts, probas, routes, mask, interp=true, obsviz=true)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpKeTRQU", 0x0000000000000036, 2.0, nothing)

In [22]:
include("masking.jl")
include("render_helpers.jl")

In [66]:
fieldnames(LaneTag)

2-element Array{Symbol,1}:
 :segment
 :lane   

In [99]:
s = h.state_hist[2]
b = h.belief_hist[2]
ai = h.ainfo_hist[2]
a = h.action_hist[2]
# println(ai[1])
# println(ai[2])
# println(a)
# println(safe_actions(pomdp, rand_pol.mask, s, 101, 2))
# println(safe_actions(pomdp, rand_pol.mask, b))
action(rand_pol, b)
# for veh in s
#     println(veh)
# end
s_mdp = get_mdp_state(rand_pol.mask.mdp, rand_pol.pomdp, s, PED_ID, CAR_ID)
itp_ped, itp_ped_w = interpolate_pedestrian(mdp, s_mdp.ped)
for ped in itp_ped
    println(ped)
end
println(s_mdp.ped)

VehicleState(VecSE2({-7.000, -5.000}, 0.000), Frenet(RoadIndex({1, 0.000000}, {19, 1}), 0.000, 0.000, 0.000), 1.000)
VehicleState(VecSE2({-5.000, -5.000}, 0.000), Frenet(RoadIndex({1, 0.142857}, {19, 1}), 2.000, 0.000, 0.000), 1.000)
VehicleState(VecSE2({-6.079, -5.423}, 0.000), Frenet(RoadIndex({1, 0.065755}, {19, 1}), 0.921, -0.423, 0.000), 1.000)


In [78]:
s_mdp.ped

VehicleState(VecSE2({-28.000, -29.100}, 0.000), Frenet(RoadIndex({1, 0.000000}, {18, 1}), 0.000, 22.000, -1.571), 0.000)

In [68]:
model = h.info_hist[57][2]
model.a


LoadError: [91mBoundsError: attempt to access 44-element Array{Any,1} at index [57][39m

In [64]:
[m.a for m in model.crosswalk_drivers]
m = model.intersection_driver
m.priority

true

In [61]:
AutomotivePOMDPs.engaged(m, state_hist[57], env.roadway, 2)

true

In [63]:
# AutomotivePOMDPs.engaged(model, state_hist[25], env.roadway, 2)
observe!(model, state_hist[25], env.roadway, 2)

AutomotivePOMDPs.UrbanDriver
  a: AutomotivePOMDPs.LonAccelDirection
  navigator: AutomotivePOMDPs.RouteFollowingIDM
  intersection_driver: AutomotivePOMDPs.TTCIntersectionDriver
  crosswalk_drivers: Array{AutomotivePOMDPs.CrosswalkDriver}((3,))
  debug: Bool false


In [198]:
sc = state_hist[1]
b = h.belief_hist[1]
safe_actions(pomdp, mask, sc, 101, 2)

4-element Array{AutomotivePOMDPs.UrbanAction,1}:
 AutomotivePOMDPs.UrbanAction(-4.0)
 AutomotivePOMDPs.UrbanAction(-2.0)
 AutomotivePOMDPs.UrbanAction(0.0) 
 AutomotivePOMDPs.UrbanAction(2.0) 

In [24]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, rand_pol, n_ep=10000, max_steps=100, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:03m

13856.646990 seconds (39.01 G allocations: 1.467 TiB, 6.75% gc time)
Summary for 

Progress: 100%|█████████████████████████████████████████| Time: 3:50:56


10000 episodes: 
Average reward: 0.037 
Average # of steps: 76.584 
Average # of violations: 2.460 


In [146]:
@showprogress for ep=1:10000
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initialstate(pomdp, rng)
    o0 = generate_o(pomdp, s0, rng)
    up = FastPreviousObservationUpdater{UrbanObs}()
    b0 = initialize_belief(up, o0)
    hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0)
    if sum(hist2.reward_hist .< 0.) != 0.
        println("Crash")
        break
    end
end

Progress:  66%|███████████████████████████              |  ETA: 0:29:44

Crash


Progress: 100%|█████████████████████████████████████████| Time: 0:56:44


## Trained Policy

In [10]:
include("masked_dqn.jl")

In [15]:
problem_file="joint-log/log7/problem.jld"
weights_file="joint-log/log7/weights.jld"
solver = jldopen(problem_file, "r") do file
    read(file, "solver")
end
env_ = POMDPEnvironment(pomdp)
graph = TensorFlow.Graph()
train_graph = DeepQLearning.build_graph(solver, env_, graph)
policy = DeepQLearning.restore_policy(env_, solver, train_graph, weights_file);

2018-09-21 17:10:11.379149: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


In [16]:
masked_policy = MaskedDQNPolicy(pomdp, policy, mask);

In [17]:
function POMDPToolbox.action_info(policy::MaskedDQNPolicy, s)
    return action(policy, s), safe_actions(policy.problem, policy.mask, s)
end

In [18]:
hr = HistoryRecorder(rng=rng, max_steps=400)
s0 = initialstate(pomdp,rng)
o0 = generate_o(pomdp, s0, rng)
up = FastPreviousObservationUpdater{UrbanObs}()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, masked_policy, up, b0, s0);

  2.630111 seconds (2.36 M allocations: 108.711 MiB, 2.78% gc time)


In [19]:
include("render_helpers.jl")

In [21]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
safe_acts = h.ainfo_hist
push!(action_hist, UrbanAction(NaN))
push!(safe_acts, [UrbanAction(NaN)])
duration, fps, render_hist = animate_states(pomdp, state_hist, action_hist, safe_acts, mask, interp)
speed_factor = 2
film = roll(render_hist, fps = speed_factor*fps, duration = duration/speed_factor)

LoadError: [91mMethodError: no method matching animate_states(::AutomotivePOMDPs.UrbanPOMDP, ::Array{Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},1}, ::Array{AutomotivePOMDPs.UrbanAction,1}, ::Array{Any,1}, ::MDPModelChecking.SafetyMask{PedCar.PedCarMDP,DiscreteValueIteration.ValueIterationPolicy})[0m
Closest candidates are:
  animate_states(::AutomotivePOMDPs.UrbanPOMDP, ::Array{Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},1}, ::Array{AutomotivePOMDPs.UrbanAction,1}, ::Array{Any,1}; overlays, cam) at /mnt/c/Users/Maxime/OneDrive - Leland Stanford Junior University/Research/AutomotiveSafeRL/render_helpers.jl:127
  animate_states(::AutomotivePOMDPs.UrbanPOMDP, ::Array{Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},1}, ::Array{AutomotivePOMDPs.UrbanAction,1}, ::Array{Any,1}, [91m::JointMask[39m; o

In [85]:
function MDPModelChecking.safe_actions(pomdp::UrbanPOMDP, mask::SafetyMask{PedCarMDP, P}, s::UrbanState, ped_id, car_id) where P <: Policy
    s_mdp = get_mdp_state(mask.mdp, pomdp, s, ped_id, car_id)
    itp_states, itp_weights = interpolate_state(mask.mdp, s_mdp)
    action_space = actions(mask.mdp)
    # compute risk vector
    p_sa = zeros(n_actions(mask.mdp))
    for (i, ss) in enumerate(itp_states)
        vals = value_vector(mask.policy, ss)
        p_sa += itp_weights[i]*vals
    end
#     println(p_sa)
    safe_acts = UrbanAction[]
    sizehint!(safe_acts, n_actions(mask.mdp))
    if maximum(p_sa) <= mask.threshold
        push!(safe_acts, action_space[indmax(p_sa)])
    else
        for (j, a) in enumerate(action_space)
            if p_sa[j] > mask.threshold
                push!(safe_acts, a)
            end
        end
    end
    return safe_acts
end

safe_actions(pomdp, mask, state_hist[1], 101, 2)

4-element Array{AutomotivePOMDPs.UrbanAction,1}:
 AutomotivePOMDPs.UrbanAction(-4.0)
 AutomotivePOMDPs.UrbanAction(-2.0)
 AutomotivePOMDPs.UrbanAction(0.0) 
 AutomotivePOMDPs.UrbanAction(2.0) 

In [38]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, masked_policy, n_ep=1000, max_steps=400, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

Progress: 100%|█████████████████████████████████████████|  ETA: 0:00:01

551.768113 seconds (3.30 G allocations: 126.089 GiB, 15.24% gc time)
Summary for 1000 episodes: 
Average reward: 0.019 
Average # of steps: 114.100 
Average # of violations: 0.100 


Progress: 100%|█████████████████████████████████████████| Time: 0:09:12
